In [ ]:
import io
import numpy as np
import random
import torch 
import torch.utils.data

import os
os.chdir('/data/home/Voxnet/Projet') 


def txt_to_tensordataset(fichier='train2.txt',num_classes=num_classes): # for example: txt_to_tensodataset('train2.txt',10) to load augmented trainset for 10 classes of objects.
    Fichier = open(fichier,'r') 
    train=[] #list of arrays
    string=Fichier.readline()
    labels=[]
    trainlabel=[]
    while True:
        #new matrix
        if len(string)==3:
            label=int(string[1])
            if (label in labels)==False:
                print(label)
            labels.append(label)
        elif len(string)==4 : 
            label=int(string[1])*10+int(string[2])
            if (label in labels)==False:
                print(label)
            labels.append(label)
        temp=-np.ones((32,32,32)) # binary occupancy model : 0 if unnocupied, 1 if occupied.
                                  # we substract 0.5 and multiply by 2: -1 if unnocupied, 1 if occupied
        string=Fichier.readline()
        if label==num_classes:
            break   
        while string[0]!='#': 
            # the txt file is as following: #label followed by the coordinates of non zeroes ,each line x,y,z
            #the end of the file is marked by #40.
            s=string
            string=Fichier.readline()
            if s[1]==',':
                x=int(s[0])
                s=s[1:]
                s=s[1:]
            else :
                x=int(s[0])*10+int(s[1])
                s=s[1:]
                s=s[1:]
                s=s[1:]
            if s[1]==',':
                y=int(s[0])
                s=s[1:]
                s=s[1:]
            else :
                y=int(s[0])*10+int(s[1])
                s=s[1:]
                s=s[1:]
                s=s[1:]
            if len(s)==2:
                z=int(s[0])
            else:
                z=int(s[0])*10+int(s[1])
            temp[x+1,y+1,z+1]=1
        train.append(temp)
        trainlabel.append(label)
    Fichier.close()
    train = np.asarray(train)
    trainlabel=np.asarray(trainlabel)
    train = torch.from_numpy(train)
    tensor_target = torch.from_numpy(trainlabel)
    trainset = torch.utils.data.TensorDataset(train, tensor_target)
    return(train,trainset)



In [ ]:
import matplotlib.pyplot as plt
cuda=True
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
def accuracy(net, test_loader, cuda=cuda):
    net.eval()
    correct = 0
    total = 0
    loss = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            if cuda:
                images = images.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.LongTensor)
            outputs = net(images)
            # loss+= criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # if total > 100:
            # break
    net.train()
    print('Accuracy of the network on the loader : %d %%' % (100 * correct / total))
    # return (100.0 * correct / total, loss/total)
    return 100.0 * correct / total

def train(net, optimizer, train_loader, test_loader, loss,  n_epoch = 5,
          train_acc_period = 100,
          test_acc_period = 5,
          cuda=cuda):
    total = 0
    for epoch in range(n_epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs
            inputs, labels = data
            if cuda:
                inputs = inputs.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.LongTensor)
            # print(inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
          
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            # print statistics
            running_loss = 0.33*loss.item()/labels.size(0) + 0.66*running_loss
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()/labels.size(0)
            running_acc = 0.3*correct + 0.66*running_acc
            if i % train_acc_period == train_acc_period-1:
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
                print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
                running_loss = 0.0
                total = 0
            
        if epoch % test_acc_period == test_acc_period-1:
            cur_acc, cur_loss = accuracy(net, test_loader, cuda=cuda)
            print('[%d] loss: %.3f' %(epoch + 1, cur_loss))
            print('[%d] acc: %.3f' %(epoch + 1, cur_acc))
           
    print('Finished Training')

In [ ]:

def trace_points(volume):    
    [x,y,z]= volume.nonzero()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(-x, y, z, zdir='z')


def trace(volume):# prend un np maxi array avec des 0 et des 1 et voxelise
    fig, _ = plt.subplots(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(volume, edgecolor="black")
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot3DVoxel(voxels):
    fig = plt.figure('Point Cloud 3D Voxelization')
    fig, _ = plt.subplots(figsize=(10, 10))
    plt3d = fig.gca(projection='3d')

    occupied = (voxels == 1)
    free = (voxels == 0)

    # # set the colors of each object
    colors = np.zeros(voxels.shape + (4,))
    colors[free] = [0.1, 0.1, 0.1, 0.1]
    colors[occupied] = [0.8, 0.8, 0.8, 1.0]

    # setting camera angle
    plt3d.set_xlabel('X', fontsize=9)
    plt3d.set_ylabel('Y', fontsize=9)
    plt3d.set_zlabel('Z', fontsize=9)
    plt3d.set_xlim3d(0, voxels.shape[0])
    plt3d.set_ylim3d(0, voxels.shape[1])
    plt3d.set_zlim3d(0, voxels.shape[2])

    # and plot everything
    # plt3d.voxels(occupied | free, facecolors=colors, edgecolor='k', linewidth=0.8)
    plt3d.voxels(occupied, facecolors=colors, edgecolor='k', linewidth=0.8)

    dx, dy, dz = voxels.shape
    x = y= z = 0
    xx = [x, x, x+dx, x+dx, x]
    yy = [y, y+dy, y+dy, y, y]
    kwargs = {'alpha': 1, 'color': 'white'}
    plt3d.plot3D(xx, yy, [z]*5, **kwargs)
    plt3d.plot3D(xx, yy, [z+dz]*5, **kwargs)
    plt3d.plot3D([x, x], [y, y], [z, z+dz], **kwargs)
    plt3d.plot3D([x, x], [y+dy, y+dy], [z, z+dz], **kwargs)
    plt3d.plot3D([x+dx, x+dx], [y+dy, y+dy], [z, z+dz], **kwargs)
    plt3d.plot3D([x+dx, x+dx], [y, y], [z, z+dz], **kwargs)

    plt.show()